<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo II: Vectores Palabra (Word Embeddings)

# Word Embeddings



## Motivación

### El gran problema de Bag of Words

Pensemos en estas 3 frases como documentos:

- $doc_1$: `¡Buenísimo el croissant!`
- $doc_2$: `¡Estuvo espectacular ese pan francés!`
- $doc_3$: `!Buenísima esa pintura!`

Sabemos $doc_1$ y $doc_2$ hablan de lo mismo 🍞🍞👌 y que $doc_3$ 🎨 no tiene mucho que ver con los otros.

Supongamos que queremos ver que tan similares son ambos documentos. 

Para esto, generamos un modelo `Bag of Words` sobre el documento. Es decir, transformamos cada palabra a un vector one-hot y luego los sumamos por documento. 

Además, omitimos algunas stopwords y consideramos pan frances como un solo token.

$$v = \{buenísima, croissant, estuvo, espectacular, pan\ francés, pintura\}$$

Entonces, el $\vec{doc_1}$ quedará:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} =
  \begin{bmatrix}1 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix}$$

El $\vec{doc_2}$ quedará:

$$\begin{bmatrix}0 \\ 0 \\ 1 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1\\ 0\end{bmatrix} = 
  \begin{bmatrix}0 \\ 0 \\ 1 \\ 1 \\ 1\\ 0\end{bmatrix}$$

Y el $\vec{doc_3}$: 

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix} =
  \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix}$$



**¿Cuál es el problema?**

`buenísima` $\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `espectacular` $ \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$ representan ideas muy similares. Por otra parte, sabemos que `croissant` $\begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `pan francés` $\begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1 \\0\end{bmatrix}$ se refieren al mismo objeto. Pero en este modelo, estos **son totalmente distintos**. Es decir, los vectores de las palabras que `buenísima` y `espectacular` son tan distintas como `croissant` y `pan francés`. Esto evidentemente, repercute en la calidad de los modelos que creamos a partir de nuestro Bag of Words.

Ahora, si queremos ver que documento es mas similar a otro usando distancia euclidiana, veremos que:

$$d(doc_1, doc_2) = 2.236$$
$$d(doc_1, doc_3) = 1.414$$

Es decir, $doc_1$ se parece mas a $doc_3$ aunque nosotros sabemos que $doc_1$ y $doc_2$ nos están diciendo lo mismo!


Nos gustaría que eso no sucediera. Que existiera algún método que nos permitiera hacer que palabras similares tengan representaciones similares. Y que con estas, representemos mejor a los documentos.


--------------------

### Hipótesis Distribucional

Estamos buscando algún enfoque que nos permita representar las palabras de forma no aislada, si no como algo que además capture el significado de esta.

Pensemos un poco en la **hipótesis distribucional**. Esta plantea que:

    "Palabras que ocurren en contextos iguales tienden a tener significados similares." 

O equivalentemente,

    "Una palabra es caracterizada por la compañía que esta lleva."

Esto nos puede hacer pensar que podríamos usar los contextos de las palabras para generar vectores que describan mejor dichas palabras: en otras palabras, los `Distributional Vectors`.

--------------------


### Word-Context Matrices

Es una matriz donde cada celda $(i,j)$ representa la co-ocurrencia entre una palabra objetivo/centro $w_i$ y un contexto $c_j$. El contexto son las palabras dentro de ventana de tamaño $k$ que rodean la palabra central. 

Cada fila representa a una palabra a través de su contexto. Como se puede ver, ya no es un vector one-hot, si no que ahora contiene mayor información.

El tamaño de la matriz es el tamaño del vocabulario $V$ al cuadrado. Es decir $|V|*|V|$.

<img src="./Figures/distributionalSocher.png" alt="Word-context matrices" style="width: 400px;"/>


**Problema: Creada a partir de un corpus respetable, es gigantezca**. 

Por ejemplo, para $|v| = 100.000$, la matriz tendrá $\frac{100000 * 100000 * 4}{10^9} = 40gb $.

- Es caro mantenerla en memoria 
- Los clasificadores no funcionan tan bien con tantas dimensiones (ver [maldición de la dimensionalidad](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)).

¿Habrá una mejor solución?

---------------------

### Word Embeddings


La idea principal de los Word Embeddings es crear representaciones vectoriales densas y de baja dimensionalidad $(d << |V|)$ de las palabras a partir de su contexto.  Para esto, se usan distintos modelos que emplean redes neuronales *shallow* o poco profundas.

Volvamos a nuestro ejemplo anterior: `buenísima` y `espectacular` ocurren muchas veces en el mismo contexto, por lo que los embeddings que los representan debiesen ser muy similares... :

`buenísima` $\begin{bmatrix}0.32 \\ 0.44 \\ 0.92 \\ .001 \end{bmatrix}$ y `espectacular` $\begin{bmatrix}0.30 \\ 0.50 \\ 0.92 \\ .002 \end{bmatrix}$ versus `croissant`  $\begin{bmatrix}0.77 \\ 0.99 \\ 0.004 \\ .1 \end{bmatrix}$ el cuál es claramente distinto.


Pero, **¿Cómo capturamos el contexto dentro de nuestros vectores?**

- Dependerá del modelo que utilizemos.


##### Word2vec y Skip-gram

Word2Vec es probablemente el paquete de software mas famoso para crear word embeddings. Este nos provee herramientas para crear distintos tipos de modelos, tales como `Skip-Gram` y `Continuous Bag of Word (CBOW)`. En este caso, solo veremos `Skip-Gram`.

**Skip-gram** es una task auxiliar con la que crearemos nuestros embeddings. Esta consiste en que por cada palabra del dataset, predigamos las palabras de su contexto (las palabras presentes en ventana de algún tamaño $k$).

Para resolverla, usaremos una red de una sola capa oculta. Los pesos ya entrenados de esta capa serán los que usaremos como embeddings.

#### Detalles del Modelo

- Como dijimos, el modelo será una red de una sola capa. La capa oculta tendrá una dimensión $d$ la cual nosotros determinaremos. Esta capa no tendrá función de activación. Sin embargo, la de salida si, la cual será una softmax.

- El vector de entrada, de tamaño $|V|$, será un vector one-hot de la palabra que estemos viendo en ese momento.

- La salida, también de tamaño $|V|$, será un vector que contenga la distribución de probabilidad de que cada palabra del vocabulario pertenezca al contexto de la palabra de entrada.

- Al entrenar, se comparará la distribución de los contextos con la suma de los vectores one-hot del contexto real.

<img src="./Figures/skip_gram_net_arch.png" alt="Skip Gram" style="width: 600px;"/>

Nota: Esto es computacionalmente una locura. Por cada palabra de entrada, debemos calcular la probabilidad de aparición de todas las otras. Imaginen el caso de un vocabulario de 100.000 de palabras y de 10000000 oraciones...

La solución a esto es modificar la task a *Negative Sampling*. Esta transforma este problema de $|V|$ clases a uno binario.

### La capa Oculta y los Embeddings

Al terminar el entrenamiento, ¿Qué nos queda en la capa oculta?

Una matriz de $v$ filas por $d$ columnas, la cual contiene lo que buscabamos: Una representación continua de todas las palabras de nuestro vocabualrio.  

**Cada fila de la matriz es un vector que contiene la representación continua una palabra del vocabulario.**


<img src="./Figures/word2vec_weight_matrix_lookup_table.png" alt="Capa Oculta 1" style="width: 400px;"/>

¿Cómo la usamos eficientemente?

Simple: usamos los mismos vectores one-hot de la entrada y las multiplicamos por la matriz:

<img src="./Figures/matrix_mult_w_one_hot.png" alt="Skip Gram" style="width: 400px;"/>






## Entrenar nuestros Embeddings

Para entrenar nuestros embeddings, usaremos el paquete gensim. Este trae una muy buena implementación de `word2vec`.




In [2]:
# install word2vec
# !pip install gensim

### Cargar el dataset y limpiar



### Extracción de Frases

Para crear buenas representaciones, es necesario tambien encontrar conjuntos de palabras que por si solas no tengan mayor significado (como `nueva` y `york`), pero que juntas que representen ideas concretas (`nueva york`). 

Para esto, usaremos el primer conjunto de herramientas de `gensim`: `Phrases` y `Phraser`.

In [1]:
#La condición para que sean considerados es que aparezcan por lo menos 100 veces repetidas.



Ahora, usamos `Phraser` para re-tokenizamos el corpus con los bigramas encontrados. Es decir, juntamos los tokens separados que detectamos como frases.

### Definir el modelo



Primero, como es usual, creamos el modelo. En este caso, usaremos uno de los primero modelos de embeddings neuronales: `word2vec`

Algunos parámetros importantes:

- `min_count`: Ignora todas las palabras que tengan frecuencia menor a la indicada.
- `window` : Tamaño de la ventana. Usaremos 4.
- `size` : El tamaño de los embeddings que crearemos. Por lo general, el rendimiento sube cuando se usan mas dimensiones, pero después de 300 ya no se nota cambio. Ahora, usaremos solo 200.
- `workers`: Cantidad de CPU que serán utilizadas en el entrenamiento.

### Construir el vocabulario

Para esto, se creará un conjunto que contendrá (una sola vez) todas aquellas palabras que aparecen mas de `min_count` veces. 

### Entrenar el Modelo

A continuación, entenaremos el modelo. 
Los parámetros que usaremos serán: 

- `total_examples`: Número de documentos.
- `epochs`: Número de veces que se iterará sobre el corpus.

Es recomendable que tengan instalado `cpython` antes de continuar. Aumenta bastante la velocidad de entrenamiento.


###  Guardar y cargar el modelo

Para ahorrar tiempo, usaremos un modelo preentrenado.

## Tasks: Palabras mas similares y Analogías

### Palabras mas similares

Tal como dijimos anteriormente, los embeddings son capaces de codificar toda la información contextual de las palabras en vectores.

Y como cualquier objeto matemático, estos pueden operados para encontrar ciertas propiedades. Tal es el caso de las  encontrar las palabras mas similares, lo que no es mas que encontrar los n vecinos mas cercanos del vector.  